In [ ]:
# Install https://github.com/nttcslab/msm-mae for RuntimeMAE
from msm_mae.runtime import RuntimeMAE
import torch 
import torchaudio

device = torch.device('cuda')

# Create a model with pretrained weights.
runtime = RuntimeMAE(weight_file='80x512p16x16_paper/checkpoint-100.pth')
runtime = runtime.to(device)


def process_audio(fname):
    audio, sr = torchaudio.load(filepath=fname)
    audio = audio.to(device)
    if(audio.shape[0] == 2):
        audio = audio[0:1, :] + audio[1:2, :]
    frame_level = runtime.encode(audio)

    # This example ends up with frame-level 3840-d feature vectors for 63 time frames.
    # The `frame_level` will have a size of torch.Size([3, 63, 3840]).
    # print(frame_level.shape)

    #  You can get clip-level features by taking average of time franes.
    # The `clip_level` will have a size of torch.Size([3, 3840])
    clip_level = torch.mean(frame_level, dim=1)
    return clip_level



def fill(meldir, buckets):
    for l in open(meldir + '.csv'):
        label = int(l[-2])
        if(l[-3:-1] == '10'):
            id = (l[len('/home/swli2/slowfast/data/video'):-len('_final.mp4 xx\n')])
            label = 10
        else:
            id = (l[len('/home/swli2/slowfast/data/video'):-len('_final.mp4 x\n')])
        # print(label, meldir)
        

        if(label == 0):
            filename = '../audio_features/standardized_split_audio/'+meldir+'/'+(id)+'n.wav'
            potential = process_audio(filename)
            torch.save(potential, '../audio_features/mae_filtered/'+meldir+'/'+(id)+'n.pt')
            
        else:
            filename = '../audio_features/standardized_split_audio/'+meldir+'/'+(id)+'y.wav'
            potential = process_audio(filename)
            torch.save(potential, '../audio_features/mae_filtered/'+meldir+'/'+(id)+'y.pt')
            

fill('train', 5)
fill('test', 5)